This notebook creates the haussdorf metrics we also use in some experiments.

In [16]:
import os
import sys
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import nibabel as nib
module_path = os.path.abspath(os.path.join('..'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

from nnunet.evaluation.metrics import *
from util import management as mana
from util import constants as con

## zelf invullen
hd_metric = hausdorff_distance

c:\Users\ikke_\OneDrive\Documenten\Thesis\MasterThesis\Code


Define some useful functions

In [28]:
thesis_dir = "C:\\Users\\ikke_\\OneDrive\\Documenten\\Thesis"
filenames = {hausdorff_distance: "haussdorfs.txt", hausdorff_distance_95: "haussdorf95s.txt", dice: "own_dice.txt"}

def get_dirs(task, config, trainer, task2 = None):
    if task2 == None:
        task2 = task
    data_dir = f"C:/Users/ikke_/OneDrive/Documenten/Thesis/Data/nnUNet_raw_data_base/nnUNet_raw_data/Task{task}"
    
    res_dir = os.path.join(thesis_dir, "Results", str(task), config, trainer)
    cv_dir = os.path.join(res_dir, "cv_niftis_postprocessed")
    ts_dir = os.path.join(res_dir, "inference", str(task2), "imagesTs")

    lab_ts = os.path.join(data_dir, "labelsTs")
    lab_tr = os.path.join(data_dir, "labelsTr")

    return cv_dir, ts_dir, lab_ts, lab_tr

def get_dirs_p16(batch, task, config, trainer):
    data_dir = f"C:/Users/ikke_/OneDrive/Documenten/Thesis/Data/nnUNet_raw_data_base/nnUNet_raw_data"
    batch_dir = f"{data_dir}/batch{batch}"
    
    # test cases
    lab_ts = os.path.join(batch_dir, "labels")
    seg_ts = os.path.join(batch_dir, f"labels_{task}")

    # validation cases
    seg_cv = os.path.join(thesis_dir, "Results", str(task), config, trainer, "cv_niftis_postprocessed")
    lab_cv = f"{data_dir}/Task{task}/labelsTr"

    return lab_ts, seg_ts, seg_cv, lab_cv

def calculate_hds(segmentation_path, label_path):
    files = [i for i in os.listdir(segmentation_path) if i.endswith("nii.gz")]
    print(files)
    hds = []
    for file in files:
        # load seg and label
        seg_path = os.path.join(segmentation_path, file)
        lab_path = os.path.join(label_path, file)
        seg = np.array(nib.load(seg_path).dataobj)
        lab = np.array(nib.load(lab_path).dataobj)

        # compute HD
        hd = hd_metric(seg, lab)
        hds.append(hd)

    # write results
    fp = open(os.path.join(segmentation_path, filenames[hd_metric]), "w")
    for item in hds:
        # write each item on a new line
        fp.write("%s\n" % item)
    fp.write(f"\nMean: {np.nanmean(hds)}")
    fp.write(f"\nStd: {np.std(hds)}")
    fp.close()

Calculate all the HD metrics

In [ ]:
# NIH and MSD and BCV architecture experiment
# for task in [con.TASK_501, con.TASK_510, con.TASK_525]:
#     for trainer in [con.CLASSIC, con.HYBRID2, con.HYBRID2LR]:
#         cv_dir, ts_dir, lab_ts, lab_tr = get_dirs(task, con.CONFIG_FULL, trainer)
#         calculate_hds(cv_dir, lab_tr)
#         calculate_hds(ts_dir, lab_ts)

# # NIH architecture to 1500 experiment
# for task in [con.TASK_501]:
#     for trainer in [con.CLASSIC_1500, con.HYBRID2_1500, con.HYBRID2LR_1500]:
#         cv_dir, ts_dir, lab_ts, lab_tr = get_dirs(task, con.CONFIG_FULL, trainer)
#         calculate_hds(cv_dir, lab_tr)
#         calculate_hds(ts_dir, lab_ts)

# # BCV learning rate experiment
# for task in [con.TASK_525, con.TASK_526, con.TASK_527]:
#     for trainer in [con.CLASSIC, con.WEIGHT01, con.WEIGHT05, con.WEIGHT09]:
#         if task == con.TASK_525 and trainer != con.CLASSIC: 
#             continue
#         cv_dir, ts_dir, lab_ts, lab_tr = get_dirs(task, con.CONFIG_FULL, trainer)
#         calculate_hds(cv_dir, lab_tr)
#         calculate_hds(ts_dir, lab_ts)

# Combined NIH/MSD dataset
# for task in [con.TASK_800]:
#     for trainer in [con.CLASSIC]:
#         for task2 in [con.TASK_501, con.TASK_510]:
#             cv_dir, ts_dir, lab_ts, lab_tr = get_dirs(task, con.CONFIG_FULL, trainer, task2)
#             calculate_hds(lab_ts, ts_dir)

# # doortrainen tot 1500 experiment
# for task in [con.TASK_501]:
#     for trainer in [con.CLASSIC_1500, con.HYBRID2_1500, con.HYBRID2LR_1500]:
#         cv_dir, ts_dir, lab_ts, lab_tr = get_dirs(task, con.CONFIG_FULL, trainer)
#         calculate_hds(cv_dir, lab_tr)
#         calculate_hds(ts_dir, lab_ts)

# # Combined NIH/MSD dataset
# for task in [con.TASK_800]:
#     for trainer in [con.CLASSIC]:
#         for task2 in [con.TASK_510]:
#             cv_dir, ts_dir, lab_ts, lab_tr = get_dirs(task, con.CONFIG_FULL, trainer, task2)
#             lab_ts = f"C:/Users/ikke_/OneDrive/Documenten/Thesis/Data/nnUNet_raw_data_base/nnUNet_raw_data/Task510/labelsTs"
#             calculate_hds(lab_ts, ts_dir)

# #for task in [con.TASK_600, con.TASK_601]:
# for task in [con.TASK_601]:
#     for trainer in [con.CLASSIC]:
#         # calculate amos itself
#         # cv_dir, ts_dir, lab_ts, lab_tr = get_dirs(task, con.CONFIG_FULL, trainer)
#         # calculate_hds(cv_dir, lab_tr)
#         # calculate_hds(ts_dir, lab_ts)

#         # calculate MSD
#         cv_dir, ts_dir, lab_ts, lab_tr = get_dirs(task, con.CONFIG_FULL, trainer, con.TASK_510)
#         lab_ts = f"C:/Users/ikke_/OneDrive/Documenten/Thesis/Data/nnUNet_raw_data_base/nnUNet_raw_data/Task510/labelsTs"
#         calculate_hds(ts_dir, lab_ts)

# hd_metric = hausdorff_distance
# for task in [con.TASK_600, con.TASK_601]:
#     for trainer in [con.CLASSIC]:
#         # calculate amos itself
#         # cv_dir, ts_dir, lab_ts, lab_tr = get_dirs(task, con.CONFIG_FULL, trainer)
#         # calculate_hds(cv_dir, lab_tr)
#         # calculate_hds(ts_dir, lab_ts)

#         # calculate MSD
#         cv_dir, ts_dir, lab_ts, lab_tr = get_dirs(task, con.CONFIG_FULL, trainer, con.TASK_510)
#         lab_ts = f"C:/Users/ikke_/OneDrive/Documenten/Thesis/Data/nnUNet_raw_data_base/nnUNet_raw_data/Task510/labelsTs"
#         calculate_hds(ts_dir, lab_ts)

# # P16 batch 1
# for task in [con.TASK_610, con.TASK_611]:
#         lab_ts, seg_ts, seg_cv, lab_cv = get_dirs_p16(1, task, con.CONFIG_FULL, con.CLASSIC)
#         # calculate HDs on batch 1
#         if task != con.TASK_612:
#                 calculate_hds(lab_ts, seg_ts)
#         # calculate HDs on its own validation cases
#         calculate_hds(seg_cv, lab_cv)

# # P16 batch 2
# for task in [con.TASK_613]:#[con.TASK_610, con.TASK_611, con.TASK_612, con.TASK_613]:
#         lab_ts, seg_ts, seg_cv, lab_cv = get_dirs_p16(2, task, con.CONFIG_FULL, con.CLASSIC)
#          # calculate HDs on batch 2
#         if task != con.TASK_613:
#                 calculate_hds(seg_ts, lab_ts) 
#         # let 613 calculate HDs on its own validation cases
#         else:
#                 calculate_hds(seg_cv, lab_cv) 

['panc_0001.nii.gz', 'panc_0002.nii.gz', 'panc_0003.nii.gz', 'panc_0004.nii.gz', 'panc_0005.nii.gz', 'panc_0007.nii.gz', 'panc_0009.nii.gz', 'panc_0010.nii.gz', 'panc_0021.nii.gz', 'panc_0022.nii.gz', 'panc_0023.nii.gz', 'panc_0024.nii.gz', 'panc_0025.nii.gz', 'panc_0026.nii.gz', 'panc_0027.nii.gz', 'panc_0028.nii.gz', 'panc_0029.nii.gz', 'panc_0030.nii.gz', 'panc_0031.nii.gz', 'panc_0032.nii.gz', 'panc_0033.nii.gz', 'panc_0034.nii.gz', 'panc_0036.nii.gz', 'panc_0037.nii.gz', 'panc_0038.nii.gz', 'panc_0040.nii.gz']
['panc_0006.nii.gz', 'panc_0008.nii.gz', 'panc_0035.nii.gz', 'panc_0039.nii.gz']
['panc_0001.nii.gz', 'panc_0002.nii.gz', 'panc_0003.nii.gz', 'panc_0004.nii.gz', 'panc_0005.nii.gz', 'panc_0007.nii.gz', 'panc_0009.nii.gz', 'panc_0010.nii.gz', 'panc_0021.nii.gz', 'panc_0022.nii.gz', 'panc_0023.nii.gz', 'panc_0024.nii.gz', 'panc_0025.nii.gz', 'panc_0026.nii.gz', 'panc_0027.nii.gz', 'panc_0028.nii.gz', 'panc_0029.nii.gz', 'panc_0030.nii.gz', 'panc_0031.nii.gz', 'panc_0032.nii.gz